#Parte 1 : Importando Bibliotecas

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk import tokenize 
from nltk.stem import RSLPStemmer
from nltk.stem.porter import *
import heapq
import heapq as hp
import time
%matplotlib inline

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


##Lendo arquivo results csv

Eliminando caracteres com tamanho 2 ou menor e deixando todos em caixa baixa salvando na variavel "texto"

In [17]:
arquivo = pd.read_csv('results.csv')
texto = arquivo['text']

def eliminar_carac(text):
  return re.sub(r'(\W+|\d+|\b\w{0,2}\b)', ' ', text.lower())
texto = texto.apply(eliminar_carac)

texto = texto.apply(tokenize.word_tokenize, language='portuguese')
palavras = []


texto.head()

0    [juíza, federal, ivani, silva, luz, brasília, ...
1    [menos, horas, depois, juíza, federal, ivani, ...
2    [quando, determinou, que, março, uma, estratég...
3    [quando, era, pequeno, luisa, gabriel, preocup...
4    [desde, manhã, última, segunda, feira, sucesso...
Name: text, dtype: object

##Eliminando as stopwords : Usando a biblioteca nltk
###Algoritmo Build Index : Usada para uma indexação simples

In [18]:
stopwords = nltk.corpus.stopwords.words('portuguese')
def eliminar_stopwords(text):
  palavras = []
  for palavra in text:
    if(palavra not in stopwords):
      palavras.append(palavra)
  return palavras 
def build_index(arquivo):
  numero_da_lista = 0
  dic_index = {}
  
  for documento in arquivo:
    
    numero_da_lista += 1
    doc = eliminar_stopwords(documento)
    
    for palavra in doc:
      ocorrencia = doc.count(palavra)
      par = (numero_da_lista, ocorrencia)
      if palavra not in dic_index:
        dic_index[palavra] = {}
      
      if par not in dic_index[palavra]:
        
         dic_index[palavra][numero_da_lista] = ocorrencia
  return dic_index
                
index = build_index(texto)
tabela = {'palavra' : list(index.keys()),
       'doc|ocorrencia' : list(index.values()) }
df = pd.DataFrame(tabela)
df.head()

,palavra,doc|ocorrencia
0,juíza,"{1: 2, 2: 1}"
1,federal,"{1: 2, 2: 2, 3: 1, 6: 1, 7: 3, 15: 1, 24: 1, 3..."
2,ivani,"{1: 1, 2: 1}"
3,silva,"{1: 3, 2: 1, 6: 1, 14: 2, 26: 1, 73: 1, 76: 1,..."
4,luz,"{1: 3, 2: 1, 9: 1, 17: 1, 32: 2, 78: 1, 79: 1,..."


## Número de documentos que contém cada termo

In [0]:
def qdt_dois_documentos(a, b):
  docs_a = index[a].keys()
  docs_b = index[b].keys()
  count = 0
  for doc in docs_a:
    if doc in docs_b:
      count += 1
  return count  

def total_docs():
  return arquivo.text.count()

##Calculando as top-10 palavras mais associadas a cada uma dessas 5 palavras de acordo com 4 métricas

## Primeira Métrica : (MIM) Mutual Information (Informação mútua) favorecendo termos de baixa frequência

In [0]:
def mim(a, b):
  n_a = len(index[a].keys())
  n_b = len(index[b].keys())
  n_ab = qdt_dois_documentos(a, b)
  return (n_ab / (n_a * n_b))

##Segunda Métrica : (EMIM) Expected Mutual Information Measure corrige esse problema

In [0]:
def emim(a, b):
  n_a = len(index[a].keys())
  n_b = len(index[b].keys())
  n_ab = qdt_dois_documentos(a, b)
  n = total_docs()
  exp = (n * mim(a,b))
  return (n_ab * np.log(exp) if exp != 0 else 0)
  

## Terceira Métrica : (χ 2 ) Pearson’s Chi-squared

In [0]:
def chi(a, b):
  n_a = len(index[a].keys())
  n_b = len(index[b].keys())
  n_ab = qdt_dois_documentos(a, b)
  n = total_docs()
  return (n_ab - (1 / n) * n_a * n_b) ** 2 / (n_a * n_b)
   
  

##Quarta Métrica : (Dice)

In [0]:
def dice(a, b):
  n_a = len(index[a].keys())
  n_b = len(index[b].keys())
  n_ab = qdt_dois_documentos(a, b)
  return n_ab / (n_a + n_b)

## 5 Termos 

In [0]:
termos = ["política", "presidente", "bolsonaro", "maduro", "onu"]

In [0]:
def metrica_total(term, metric):
  pares = []
  for palavra in index.keys():
    if metric == 'mim':
      result = mim(termo, palavra)
    elif metric == 'emim':
      result = emim(termo, palavra)
    elif metric == 'chi':
      result = chi(termo, palavra)
    elif metric == 'dice':
      result = dice(termo, palavra)
    pares.append((palavra, result))   
  # ordenção por valor de metrica
  pares = sorted(pares, key = lambda x: x[1], reverse=True)
  return pares


termos_metricas = {}
for termo in termos:
  metricas = {}
  metricas['mim'] = metrica_total(termo, 'mim')
  metricas['emim'] = metrica_total(termo, 'emim')
  metricas['chi'] = metrica_total(termo, 'chi')
  metricas['dice'] = metrica_total(termo, 'dice')
  termos_metricas[termo] = metricas


In [0]:
def tabela_metricas(termo):
  mim = [k for k,_ in termos_metricas[termo]['mim']]
  emim = [k for k,_ in termos_metricas[termo]['emim']]
  chi = [k for k,_ in termos_metricas[termo]['chi']]
  dice = [k for k,_ in termos_metricas[termo]['dice']]
  tabela = {'MIM' : mim,
       'EMIM' : emim ,
       'X²' : chi    ,
       'DICE': dice    }
  df = pd.DataFrame(tabela)
  return df

In [55]:
tabela_metricas(termos[0]).head(10)

,MIM,EMIM,X²,DICE
0,comemorarem,política,política,política
1,instaurou,político,político,país
2,citando,cada,desigualdade,outros
3,alusiva,país,cientista,vez
4,compatível,bolsonaro,eleições,cada
5,desobedece,nacional,força,governo
6,ignora,governo,problemas,menos
7,interamericana,estado,eleição,pode
8,condenou,outros,estar,desde
9,magistrada,estar,economia,parte


In [48]:
tabela_metricas(termos[1]).head(10)

,MIM,EMIM,X²,DICE
0,juíza,presidente,presidente,presidente
1,ivani,bolsonaro,bolsonaro,feira
2,comemorarem,ministro,ministro,governo
3,instaurou,governo,deputado,bolsonaro
4,presidente,feira,presidência,ainda
5,citando,político,governo,sobre
6,rememorar,segurança,político,país
7,alusiva,afirmou,câmara,ser
8,compatível,deputado,vice,ministro
9,promovido,câmara,segurança,ter


In [49]:
tabela_metricas(termos[2]).head(10)

,MIM,EMIM,X²,DICE
0,juíza,bolsonaro,bolsonaro,bolsonaro
1,ivani,presidente,jair,presidente
2,comemorarem,brasil,psl,brasil
3,instaurou,governo,previdência,governo
4,jair,brasileiro,reforma,brasileiro
5,bolsonaro,ministro,planalto,ministro
6,citando,reforma,brasileiro,político
7,rememorar,político,brasil,brasileira
8,alusiva,psl,deputado,ainda
9,compatível,jair,presidente,paulo


In [50]:
tabela_metricas(termos[3]).head(10)

,MIM,EMIM,X²,DICE
0,nariño,maduro,maduro,maduro
1,abraçam,guaidó,guaidó,guaidó
2,arrepio,venezuela,chávez,chávez
3,emitidos,chávez,chavismo,chavismo
4,desejado,chavismo,venezuela,venezuela
5,desliza,hugo,interino,hugo
6,épico,interino,bolivariano,interino
7,contidas,assembleia,hugo,venezuelano
8,repleto,venezuelano,venezuelano,bolivariano
9,confidências,bolivariano,nicolás,assembleia


In [51]:
tabela_metricas(termos[4]).head(10)

,MIM,EMIM,X²,DICE
0,enriquecer,onu,onu,onu
1,controversa,áfrica,áfrica,áfrica
2,fabian,resoluções,resoluções,resoluções
3,salvioli,internacional,aparecendo,muçulmanos
4,onu,organização,preceitos,árabes
5,reverbera,internacionais,guatemala,jerusalém
6,napolitano,muçulmanos,árabe,israelenses
7,distorce,árabes,checa,alimentos
8,factual,entretanto,valas,aparecendo
9,consensuais,acordos,nuances,seguindo
